In [1]:
from janus_wraper import *
import torch
from transformers import AutoModelForCausalLM
from Janus.janus.models import MultiModalityCausalLM, VLChatProcessor
from Janus.janus.utils.io import load_pil_images

# specify the path to the model
# model_path = "/projectnb/cs598/projects/cool_proj/model"
# device = 'cuda'
model_path = '/Users/nover/models/deepseek-ai/Janus-Pro-7B'
device = 'mps'

chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)

gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True
)
gpt = gpt.to(torch.bfloat16).to(device).eval()





Python version is above 3.10, patching the collections module.
Python version is above 3.10, patching the collections module.


/Users/nover/repos/class/.venv/lib/python3.12/site-packages/transformers/models/auto/image_processing_auto.py:593: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thorou

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
text_prompt = 'Where is this image from?'

# # Load the image from a file
image = Image.open("img.jpg")
# tokenizer = vl_chat_processor.tokenizer

# Call the unified function for multimodal understanding (text output)
result = janus_pro_generate(
    chat_processor,
    gpt,
    device=device,
    input_text=text_prompt,
    input_image=image,
    output_mode="text",
)
print(result)


This image appears to be from a parade or military event, possibly in a country with a significant military presence. The tanks and the presence of a person walking in front of them suggest a formal display or procession.


In [2]:
text_prompt = 'make this image a meme'

# # Load the image from a file
image = Image.open("img.jpg")
# tokenizer = vl_chat_processor.tokenizer

# Call the unified function for multimodal understanding (text output)
result = janus_pro_generate(
    chat_processor,
    gpt,
    device=device,
    input_text=text_prompt,
    input_image=image,
    output_mode="image",
    guidance=5,
    t2i_temperature=1.0,
    num_images=2,
    # width=192,
    # height=192,
)


RuntimeError: shape '[2, 12, 12, 8]' is invalid for input of size 9216

In [5]:
# Save generated images to disk.
for idx, img in enumerate(result):
    display(img)


# Print the generated text response
print("Generated Text Response:")

Generated Text Response:


In [ ]:
%pip install pillow pyarrow

In [ ]:
%pip install matplotlib

In [ ]:
%pip install datasets

In [ ]:

from datasets import load_dataset
import io

dataset = load_dataset("array/SAT", batch_size=128)

# dataset should have a training and validation key
print(dataset.keys())
example = dataset['train'][:10] # example 10th item
print(example.keys())
for i in range(len(example['image_bytes'])):
    display(example['image_bytes'][i][0])
    print(example['question'][i])
    print(example['answers'][i])
    print(example['correct_answer'][i])

#again not working

In [ ]:
import pandas as pd
import io
from PIL import Image

df = pd.read_parquet('D:\BU_assignments\SEM2\MML\Project\Spatial-Reasoning\Dataset\SAT_val.parquet', engine='pyarrow')#change path here

example = df.iloc[10]
images = [Image.open(io.BytesIO(im_bytes)) for im_bytes in example['image_bytes']]

import matplotlib.pyplot as plt

# Display images
fig, axes = plt.subplots(1, len(images), figsize=(15, 5))

if len(images) == 1:
    axes.imshow(images[0])
    axes.axis("off")
else:
    for ax, img in zip(axes, images):
        ax.imshow(img)
        ax.axis("off")

plt.show()

question = example['question']
print(question)
answer_choices = example['answers']
print(answer_choices)
correct_answer = example['correct_answer']
print(correct_answer)

In [ ]:
def predict_all(dataset, vl_chat_processor, vl_gpt, device='cuda'):
    predictions = []

    for i in range(len(dataset['image_bytes'])):
        input_image = dataset['image_bytes'][i]
        question = dataset['question'][i]
        answer_choices = dataset['answers'][i]
        
        prediction = janus_pro_generate(
            vl_chat_processor=vl_chat_processor,
            vl_gpt=vl_gpt,
            input_text=question,
            answer_choices=answer_choices,
            input_image=input_image,
            device=device,
            output_mode="text",
            temperature=0.1,
            top_p=0.95,
            seed=42
        )

        predictions.append({
            "question": question,
            "answers": answer_choices,
            "prediction": prediction,
            "correct_answer": dataset['correct_answer'][i]
        })

        print(f"Question: {question}")
        print(f"Choices: {answer_choices}")
        print(f"Predicted Answer: {prediction}")
        print(f"Correct Answer: {dataset['correct_answer'][i]}")
        print("-" * 50)

    return predictions

predictions = predict_all(dataset['validation'], vl_chat_processor, vl_gpt)
#made a prediction function but might need some fixing @Surya